### Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from tqdm.auto import tqdm
import plotly.graph_objects as go
import sys
import os 
import warnings

warnings.filterwarnings("ignore")

### Custom

In [ ]:
sys.path.append(os.path.abspath('./PINNS/'))
sys.path.append(os.path.abspath('./PDE/'))
sys.path.append(os.path.abspath('./Losses/'))
from Poisson3D import Elliptical
from P3DELoss import P3DELoss
from IPINN import IPINN

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## Three Dimensional Elliptical Interface Problem

### Poisson with Elliptical Interface

$Ω = [−1, 1] × [−1, 1] × [−1, 1]$, $Γ_{int} = \{x : ψ(x) = 0, ψ(x) = 2x^2 + 3y^2 + 6z^2 − 1.69\}$<br/> subdivides the domain into two non-overlapping subdomains: <br/> $Ω_1 = \{x : ψ(x) ≤ 0\}$ and $Ω_2 = \{x : ψ(x) > 0\}$ <br/> The following PDE is considered:
Given $f_1 = 12, f_2 = 0, κ_1 = 2$, and $κ_2 = 60$, find $u_m$ (for $m = 1,2$) such that: <br/>

- $∇ · (κ_m∇u_m) = f_m$ in $Ω_m$
- $u_2 = Λ^d_2$ on $∂Ω^d_2$
- $[[u]] = (x + y + z) - (x^2 + y^2 + z^2)$  on  $Γ_{int}$
- $[[κ∇u]] · ∇Γ_{int} = (60 − 4x)4x + (60 − 4y)6y + (60 − 4z)12z$ on $Γ_{int}$

where $∂Ω^d_2 = \{x : x = −1|1 ∪ y = −1|1 ∪ z = −1|1\}$ is the Dirichlet boundary specified as per the following closed-form analytical solution:

$$ u(x, y, z) = \left\{ 
                \begin{array}{ll}
                        x^2 + y^2 + z^2 & \text{in } Ω_1 \\
                        x + y + z & \text{in } Ω_2 \\
                \end{array}
                \right. $$

In [ ]:
#define the global parameters
K = [2,60]
#get the exact solution
exact = Elliptical()
#get the loss function
elliptical_loss = P3DELoss(100000,1000,1000,K)

In [ ]:
#Plot the analytical solution
X, Y, Z = np.mgrid[-1:1:30j, -1:1:30j, -1:1:30j]
values = exact.equation(X,Y,Z)

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=-3,
    isomax=3,
    opacity=0.4, # needs to be small to see through all surfaces
    surface_count=20, # needs to be a large number for good volume rendering
    colorscale='Plasma'
    ))
fig.show()

In [ ]:
#Get the activation function
def cond_func(x,condition):
    if (condition == '0'):
        return F.silu(x)
    elif (condition == '1'):
        return F.tanh(x)

In [ ]:
#define the model
ipinn = IPINN(dimension = 3,hidden_size = 12,depth = 4)
#Get the loss_function
ipinn.setLoss(lambda x: elliptical_loss.loss(x,cond_func))
#Train the model
ipinn.train(iterations = 10000)

In [ ]:
#get the activations
activations = [F.silu,F.tanh]
#Plot the results
# Evaluate the models
x = torch.arange(-1, 1, 0.04)
y = torch.arange(-1, 1, 0.04)
z = torch.arange(-1, 1, 0.04)
X, Y, Z = torch.meshgrid(x, y, z)
XYZ = torch.stack((X, Y, Z), dim=-1).reshape(-1, 3).to(device)
u1, u2 = ipinn.eval(XYZ,activations)
result = torch.zeros_like(u1).cpu().numpy().reshape(X.shape)
u1 = u1.cpu().numpy().reshape(X.shape)
u2 = u2.cpu().numpy().reshape(X.shape)
mask = (2*X**2 + 3*Y**2 + 6*Z**2 - 1.69 <= 0)
result[mask] = u1[mask]
result[~mask] = u2[~mask]
exact_sol = exact.equation(X,Y,Z)
print('RMSE:',np.sqrt(np.mean(np.square(exact_sol - result))))

In [ ]:
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=result.flatten(),
    isomin=-3,
    isomax=3,
    opacity=0.2, # needs to be small to see through all surfaces
    surface_count=20, # needs to be a large number for good volume rendering
    colorscale='Plasma'
    ))
fig.show()

In [ ]:
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=(result - exact_sol).flatten(),
    isomin=-0.05,
    isomax=0.05,
    opacity=0.2, # needs to be small to see through all surfaces
    surface_count=20, # needs to be a large number for good volume rendering
    colorscale='Plasma'
    ))
fig.show()

### Results
With $12$ hidden layers and depth of $4$ the ipinn model gets an RMSE of $5.3 × 10^{-3}$ after $10000$ iterations 